In [1]:
%cd ../src
from util import *
from baselines import *

/media/alex/Data/Backup_root/Backup_docs/git/reddit-sarc/src


In [2]:
print("Lowercase (logistic regression)")
kfold_experiment(lower_pol_reader, fit_maxent_classifier, unigrams_phi_c, None, concat_phi_r, 5)

print("\nKeep case (logistic regression)")
kfold_experiment(pol_reader, fit_maxent_classifier, unigrams_phi_c, None, concat_phi_r, 5)

Lowercase (logistic regression)
Over 5 folds, F1 mean is 0.7235100493948599, stdev 0.0032811049751065606

Keep case (logistic regression)
Over 5 folds, F1 mean is 0.7264783004564327, stdev 0.0074785821268089185


In [2]:
pol_vocab = set()
for x in pol_reader():
    for r in x['responses']:
        words = nltk.word_tokenize(r)
        for w in words: pol_vocab.add(w)


In [5]:
fasttext_embeds = load_fasttext_embeddings(pol_vocab)

fasttext_embed_sum_phi_c = lambda x: embed_sum_phi_c(x, fasttext_embeds)

In [9]:
kfold_experiment(pol_reader, fit_maxent_classifier, fasttext_embed_sum_phi_c, None, concat_phi_r, 5, None, False)

Over 5 folds, F1 mean is 0.7119669668094337, stdev 0.014610461696180434


In [10]:
glove_amazon_embeds = load_glove_amazon_embeddings(pol_vocab)

glove_amazon_embed_sum_phi_c = lambda x: embed_sum_phi_c(x, glove_amazon_embeds)

In [12]:
kfold_experiment(pol_reader, fit_maxent_classifier, glove_amazon_embed_sum_phi_c, None, concat_phi_r, 5, None, False)

Over 5 folds, F1 mean is 0.6534898828167821, stdev 0.009875365521626909


If much higher-dimensional vectors are performing way worse than lower-dimensional ones, that probably needs we need (more) regularization in the logistic regression. Is it regularizing now?

In [4]:
glove_300_embeds = load_glove_embeddings(300, pol_vocab)

glove_300_embed_sum_phi_c = lambda x: embed_sum_phi_c(x, glove_300_embeds)

In [7]:
glove_100_embeds = load_embeddings(GLOVE_FILES[300], pol_vocab)
glove_100_embed_sum_phi_c = lambda x: embed_sum_phi_c(x, glove_300_embeds)

In [9]:
kfold_experiment(pol_reader, fit_maxent_classifier, glove_300_embed_sum_phi_c, None, concat_phi_r, 5, None, False)

Training and testing on fold 0
Training and testing on fold 1
Training and testing on fold 2
Training and testing on fold 3
Training and testing on fold 4
Over 5 folds, F1 mean is 0.6640358973447669, stdev 0.005537637288175176


In [3]:
for i,g in GLOVE_FILES.items():
    print("\nTrying glove-{} embeddings".format(i))
    glove_embeds = load_embeddings(g, pol_vocab)
    glove_phi_c = lambda x: embed_sum_phi_c(x, glove_embeds)
    kfold_experiment(pol_reader, fit_maxent_classifier, glove_phi_c, None, concat_phi_r, 5, None, False)


Trying glove-50 embeddings
Training and testing on fold 4
Over 5 folds, F1 mean is 0.6236102453641764, stdev 0.007193066996155605

Trying glove-100 embeddings
Training and testing on fold 4
Over 5 folds, F1 mean is 0.645956392357051, stdev 0.010119628368358082

Trying glove-200 embeddings
Training and testing on fold 4
Over 5 folds, F1 mean is 0.6578382009085976, stdev 0.009630159147488842

Trying glove-300 embeddings
Training and testing on fold 4
Over 5 folds, F1 mean is 0.6640358973447669, stdev 0.005537637288175176


In [14]:
def fit_maxent_classifier(X, y, C=1.0):
    mod = LogisticRegression(fit_intercept=True, C=C)
    mod.fit(X, y)
    return mod


In [12]:
for c in (0.0005, 0.001, 0.005, 0.01, 0.05):
    print("\nTesting on c={}".format(c))
    fit_fn = lambda X, y: fit_maxent_classifier(X, y, c)
    kfold_experiment(pol_reader, fit_fn, glove_amazon_embed_sum_phi_c, None, concat_phi_r, 5, None, False)
    



Testing on c=0.0005
Training and testing on fold 4
Over 5 folds, F1 mean is 0.7213696672409735, stdev 0.007459696119512551

Testing on c=0.001
Training and testing on fold 4
Over 5 folds, F1 mean is 0.7231997407523657, stdev 0.008547787185085677

Testing on c=0.005
Training and testing on fold 4
Over 5 folds, F1 mean is 0.7160304779407103, stdev 0.007996584071930475

Testing on c=0.01
Training and testing on fold 1

KeyboardInterrupt: 